In [2]:
# Here you can play with specific components of the system.

In [1]:
# Method for searching anime via AniList API
from src.anilist_query_searcher import search_anime

search_results = search_anime(
    # search_term="Attack on titan",
    # genres=["Sports", "Ecchi"],  # Using single genre parameter
    tags=["Volleyball"],
    per_page=5
)

# # Print all results first
print("All results:")
for anime in search_results:
    print(f"- {anime['title']['english'] or anime['title']['romaji']}")
    print(f"  Genres: {', '.join(anime['genres'])}")
    print(f"  Tags: {', '.join([tag['name'] for tag in anime['tags']])}")
    print("-" * 60)

All results:
- HAIKYU!!
  Genres: Comedy, Drama, Sports
  Tags: Volleyball, School Club, Male Protagonist, Primarily Male Cast, Ensemble Cast, Shounen, Primarily Teen Cast, School, Coming of Age, Fitness, Cute Boys Doing Cute Things, Kuudere
------------------------------------------------------------
- HAIKYU!! 2nd Season
  Genres: Comedy, Drama, Sports
  Tags: Volleyball, School Club, Ensemble Cast, Primarily Teen Cast, Primarily Male Cast, Shounen, Male Protagonist, Coming of Age, School, Tomboy
------------------------------------------------------------
- HAIKYU!! 3rd Season
  Genres: Comedy, Drama, Sports
  Tags: Volleyball, School Club, Shounen, Male Protagonist, Primarily Male Cast, Primarily Teen Cast, Ensemble Cast, School, Coming of Age
------------------------------------------------------------
- HAIKYU!! TO THE TOP
  Genres: Comedy, Drama, Sports
  Tags: Volleyball, Male Protagonist, Shounen, School Club, Primarily Male Cast, Ensemble Cast, School, Coming of Age, Primaril

In [2]:
# Stage 1 - Request Parser
from crewai import Agent, Task, LLM
import os
from src.request_parser import AnimeSearchParams, OFFICIAL_GENRES

# Initialize LLM (using GPT-4o from OpenAI)
llm = LLM(
    model="gpt-4.1",
    api_key=os.getenv('OPENAI_API_KEY'),
    max_tokens=2000,
    temperature=0
)

mapper = Agent(
    name="AniListRequestMapper",
    role="Filter extractor",
    goal="Return only the filters in the user's request, as minimal JSON.",
    backstory="An anime librarian who knows the difference between genres and tags.",
    allow_delegation=False,
    llm=llm
)

map_request = Task(
    description=(
        "USER_REQUEST:\n"
        "{user_request}\n\n"
        "Produce **one JSON object** that validates against AnimeSearchParams.\n"
        "Rules:\n"
        f"• Only items in this list may appear in `genres`: {OFFICIAL_GENRES}.\n"
        "• Any other descriptive phrase (moods, sub‑genres like 'School Life', "
        "adjectives like 'Wholesome') must go into `tags`.\n"
        "• Title‑case every word (e.g. 'school‑life' → 'School Life').\n"
        "Don't necessarily include genres unless they are specified in the request.\n"
        "• Omit every key whose value would be null.\n\n"
        "Example:\n"
        "USER_REQUEST:  Recommend a dark fantasy from 2020.\n"
        "OUTPUT: {\"genres\": [\"Fantasy\"], \"tags\": [\"Dark\"], \"year\": 2020}"
    ),
    expected_output="A JSON dict with only the mentioned filters, no nulls.",
    output_json=AnimeSearchParams,
    agent=mapper,
)

In [3]:
# Testing for stage 1 - Request Parser
from crewai import Crew, Process

prompt = "I want an isekai anime with some comedy"

crew = Crew(
    agents=[mapper],
    tasks=[map_request],
    process=Process.sequential,
)
crew.kickoff(inputs={"user_request": prompt})

params = {k: v for k, v in map_request.output.json_dict.items() if v is not None}
print(params)



{'genres': ['Comedy'], 'tags': ['Isekai']}


In [4]:
# Stage 2 - Anime searcher and recommender
from src.recommender import SearchAnimeTool
from crewai import Agent, Crew, Task

anime_tool = SearchAnimeTool()

anime_researcher = Agent(
    role="Anime Researcher",
    goal="Find animes that match a user query.",
    backstory="You are a seasoned anime critic.",
    tools=[anime_tool],
)

recommendation_task = Task(
    description="Using AniList, compile a list of five anime that match the query.",
    expected_output="Five recommendations with one‑sentence justifications.",
    agent=anime_researcher,
)

In [5]:
# Arming the whole crew: Stage 1 + Stage 2
crew = Crew(
    agents=[mapper, anime_researcher],
    tasks=[map_request, recommendation_task],
    process=Process.sequential,
)

In [6]:
# Testing the whole system

In [7]:
result = crew.kickoff(inputs={"user_request": "I want an anime like Akira"})
print(result)

1. **Akira (1988)** - A groundbreaking anime film that explores a dystopian future where a teenage delinquent gains psychic powers, and it perfectly captures the gritty, cyberpunk themes akin to "Akira."  
   ![Akira](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx47-4CR68arv452h.jpg)

2. **Texhnolyze (2003)** - This series delves into a dark examination of humanity's struggle for survival in a dystopian underground city, reminiscent of the oppressive atmosphere and philosophical themes presented in "Akira."  
   ![Texhnolyze](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx26-ADSztyHBNO39.jpg)

3. **Casshern Sins (2008)** - Set in a post-apocalyptic world filled with despair and existential questions, it narrates the journey of Casshern as he confronts a world plagued by death and decay, capturing the essence of Akira's tone.  
   ![Casshern Sins](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx4981-V6MbMiJUqdvP.jpg)

4. **Expelled From

In [8]:
result = crew.kickoff(inputs={"user_request": "I want an anime like ghost in the shell"})
print(result)

1. **PSYCHO-PASS** - This anime explores a dystopian future governed by a system that quantifies human emotions, drawing parallels to "Ghost in the Shell" in its themes of justice and psychological exploration. ![Cover Image](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx13601-i42VFuHpqEOJ.jpg)

2. **Vivy: Fluorite Eye’s Song** - Featuring an AI protagonist, this series tackles the complexities of consciousness and ethics in AI, resonating with the philosophical inquiries found in "Ghost in the Shell." ![Cover Image](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx128546-UIwyhuhjxmL0.jpg)

3. **Ghost in the Shell** - The original 1995 film, a classic in the cyberpunk genre, it dives into themes of identity, technology, and the nature of humanity, providing a rich storytelling experience. ![Cover Image](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx43-Y6EjeEMM14dj.png)

4. **PLUTO** - A reimagining of classic source materials, this ser

In [9]:
result = crew.kickoff(inputs={"user_request": "I want an tennis anime with some comedy"})
print(result)

1. **The Prince of Tennis** - A classic sports anime that combines intense tennis matches with comedic school life, showcasing the growth and rivalry among young players.
   ![The Prince of Tennis](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx22-yEguU9EmxkjK.png)

2. **Teekyuu** - A hilariously fast-paced comedy featuring a group of girls in a tennis club, known for its surreal humor and rapid-fire jokes that keep audiences laughing.
   ![Teekyuu](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx15125-2ngZyS4KrMqA.png)

3. **Teekyuu 2** - The second season of the beloved series continues the zany adventures of the tennis club girls, maintaining its quirky charm and offbeat humor.
   ![Teekyuu 2](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx18121-k8UaPii7p9sl.jpg)

4. **Prince of Tennis II** - A sequel that brings back beloved characters and presents new challenges in tennis, while continuing to infuse comedy into competitive sports.
